<a href="https://colab.research.google.com/github/yeaeunJi/deep_learning-/blob/main/%EC%98%A4%EC%B0%A8%EC%97%AD%EC%A0%84%ED%8C%8C%EB%B2%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 오차역전파법(backpropagation)은 가중치 매개변수의 기울기를 효율적으로 계산할 수 있음
- 오차역전파란 풀어서 설명하면 '오차를 역방향으로 전파하는 방법'이라고 할 수 있음




## 계산그래프(computational graph)
- 계산그래프를 통해서 오차역전파에 대해 시각적으로 이해할 수 있음
- 계산그래프란 계산 과정을 그래프로 나타낸 것

### 계산 그래프로 풀어보기
- 슈퍼에서 1개에 100원인 사과 2개를 구매했을 때, 지불 금액은 얼마인가(단, 소비세가 10% 부과됨)
- 슈퍼에서 한개에 100원인 사과 2개, 1개에 150원인 귤 3개를 구매했을 때, 지불 금액은 얼마인가(단, 소비세는 10% 부과됨)

- 계산 그래프를 푸는 흐름
  - 계산 그래프를 구성한 후 왼쪽에서 오른쪽으로 계산 진행(순전파)
  
  * 순전파 : 계산 그래프의 출발점에서 종착점으로의 전파
  * 역전파 : 계산 그래프의 종착점에서 출발점으로 전파

- 역전파는 미분을 계산할 때 중요한 역할을 수행함 

### 국소적 계산
- 계산 그래프의 특징 : 국소적 계산을 전파하여 최종 결과를 얻음

* 국소적 : 자신과 직접 관계된 작은 범위

  - 각 노드는 자신과 관련된 계산 외에는 관심x

- 계산 그래프의 이점 
  - 전체가 아무리 복잡해도 각 노드에서는 단순한 계산에 집중하여 문지를 단순화 가능 + 중간 계산 결과를 모두 보관할 수 있음
  - 가장 중요한 부분은 역전파를 통해 미분을 효율적으로 계산 가능
  (국소적 미분을 수행한 후 그 값을 왼쪽으로 전달)

## 단순한 계층 구현하기

In [3]:
# 곱셈 계층 구현하기
class MulLayer :
  def __init__ (self) :
    # 순전파시 입력 값을 유지하기 위해서 사용
    self.x = None
    self.y = None

  def forward(self, x, y) :
    self.x = x
    self.y = y
    out = x * y

    return out
  
  def backward(self, dout) :
    dx = dout * self.y # x와 y를 바꾼다
    dy = dout * self.x 

    return dx, dy

In [4]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [5]:
# 역전파를 통해  각 변수에 대한 미분 값 구하기
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple_price, dapple_num, dtax)

1.1 110.00000000000001 200


In [6]:
# 덧셈 계층 구현하기
class AddLayer :
  def __init__(self) :
    pass

  def forward(self, x, y) :
    out = x+y
    return out
  
  def backward(self, dout) :
    dx = dout * 1
    dy = dout * 1
    return dx, dy 

In [10]:
# 위에서 구현한 덧셈 계층과 곱셈 계층을 통해 사과 2개와 귤 3개를 사는 상황을 구현
apple = 100
apple_num = 2 
orange = 15
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 역전파
dprice  = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

269.5
110.00000000000001 2.2 3.3000000000000003 16.5 245


## 활성화 함수 계층 구현
### ReLU 계층
- 순전파 때 입력 x가 0보다 크면 역전파는 상류의 값을 그대로 하류로 보내고,
순전파 때 입력 x가 0보다 작거나 같으면 역전파는 그 값을 하류로 신호 보내지 않음(0을 보냄)



In [ ]:
## ReLu 계층 구현
class Relu :
  def __init__(self) :
    self.mask = None

  def forward(self, x) :
    self.mask = ( x <= 0)
    out = x.copy()
    out[self.mask] = 0

  def backward(self, dout) :
    dout[self.mask] = 0
    dx = dout
    return dx

In [12]:
import numpy as np
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = (x <= 0 )
print(mask)



[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


### Sigmoid 계층
- 순전파의 출력 y만으로도 역전파를 계산할 수 있음

In [ ]:
class Sigmoid :
  def __init__(self) : 
    self.out = None
  
  def forward(self, x ) :
    out = 1 / (1+np.exp(-x))
    self.out = out # 순전파의 출력을 보관 후 역전파 계산 시 사용
    return out
  
  def backward(self, dout) :
    dx = dout * (1.0 - self.out) * self.out
    return dx

## Affine/Softmax 계층 구현


In [15]:
class Affine :
  def __init__(self, W, b) :
    self.W = W
    self.b = b
    self.x = None
    self.dW = None
    self.db = None

  def forward(self, x) :
    self.x = x
    out = np.dot(x, self.W) + self.b

    return out
  
  def backward(self, dout) :
    dx = np.dot(dout, self.W.T)
    self.dW = np.dot(self.x.T, dout)
    self.db = np.sum(dout, axis = 0)

    return dx
  

### Softmax-with-Loss 계층
- 출력층에서 사용하는 소프트 맥스 함수(입력 값을 정규화(출력의 합이 1이됨)하여 출력)
- 신경망 추론 시에는 softmax 계층이 필요없으나 신경망을 학습할 때에는 Softmax 계층이 필요
- 소프트맥스 계층 + 손실 함수인 교차 엔트로피 오차도 포함한 계층을 구현해봄
- 소프트맥스의 역전파는 소프트맥스의 출력값과 정답 레이블의 차분(오차)가 앞 계층에 전해지는 것
- 회귀 출력층에서 사용하는 '항등함수'의 손실 함수로 '오차제곱합'을 이용하는 이유는 이를 사용하면 역전파의 결과가 출력값-정답인 차분으로 말끔히 떨어지기 때문

In [16]:
class SoftmaxWithLoss :
  def __init__(self):
    self.loss = None # 손실
    self.y = None # softmax의 출력
    self.t = None # 정답레이블(원-핫 벡터)

  def forward(self, x, t) :
    self.t = t
    self.y = softmax(x)
    self.loss = corss_entropy_error(self.y, self.t)
    return self.loss

  def backward(self, dout = 1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size # 역전파 시 전파하는 값을 배치의 수로 나눠서 데이터 1개당 오차를 앞 계층에 전파함